# Collecte de données


In [2]:
import os
import pandas as pd

# === Récupération du type de fichier image associé à chaque pokémon ===
lst_files = os.listdir('images/images')
lst_files = [f.split('.') for f in lst_files]
dico_files = {paire[0]: paire[1] for paire in lst_files}

#importation des données depuis un fichier csv
with open('images/pokemon.csv', 'r') as csv_file:

    # === Pré-traitement des données ===
    lines = csv_file.readlines()
    lines = lines[1:] #suppression de la première ligne (entête)
    
    for i,data in enumerate(lines):
        data = data.replace('\n', '')
        lines[i] = data.split(',')
        if len(lines[i]) == 2: #si le pokémon n'a qu'un seul type
            lines[i].append(None) #ajout d'une colonne vide pour le second type
        #récupération du type de fichier image associé   
        lines[i].append(dico_files[lines[i][0]])
        
    # === Création d'un DataFrame ===      
    df = pd.DataFrame(lines, columns=["name", "type1", "type2","img_type"])
    
    csv_file.close()

# Export du DataFrame vers un fichier JSON
df.to_json('donnees.json',orient='index')

# Étiquetage et annotation

In [32]:
from PIL import Image
import numpy
import math
import pandas as pd
from sklearn.cluster import KMeans

import Color as color

# /!\ Attention, cette cellule peut prendre plusieurs minutes à s'exécuter

# === chargement des données depuis le fichier JSON ===
df = pd.read_json('donnees.json', orient='index')
#ajout d'une colonne pour les couleurs dominantes
df['colors'] = None

# === Traitement des données ===
for pokemon in df.iterrows():
    img = Image.open(f"images/images/{pokemon[1]['name']}.{pokemon[1]['img_type']}")
    numarray = numpy.array(img, numpy.uint8)
    numarray = numarray.reshape(-1, 9)

    # récupération des couleurs dominantes
    n_clusters = 5
    clusters = KMeans(n_clusters=n_clusters, n_init=20)
    clusters.fit(numarray)

    couleurs = []
    for i in range(len(clusters.cluster_centers_)):
        couleurs.append([numpy.sum(clusters.labels_==i),"#%02x%02x%02x" % (
            math.ceil(clusters.cluster_centers_[i][0]),
            math.ceil(clusters.cluster_centers_[i][1]),
            math.ceil(clusters.cluster_centers_[i][2]),
        )])
    
    #filtrage des couleurs de fond (blanc pour jpg, noir pour png)
    couleurs = sorted(couleurs, key=lambda x: x[0], reverse=True)
    if (pokemon[1]['img_type'] == 'jpg') and (color.is_near(couleurs[0][1], '#ffffff', 5, mode='HEX')):
        couleurs.pop(0)
    elif (pokemon[1]['img_type'] == 'png') and (color.is_near(couleurs[0][1], '#000000', 5, mode='HEX')):
        couleurs.pop(0)
    else:
        couleurs.pop(-1)

    #Normalisation des valeurs de quantité de couleurs
    sum = 0
    for paire in couleurs:
        sum += paire[0]
    for paire in couleurs:
        paire[0] = int((paire[0] / sum)* 100)

    # === Ajout des couleurs dominantes au DataFrame ===
    df.at[pokemon[0], 'colors'] = couleurs


# Export du DataFrame vers le fichier JSON
df.to_json('donnees.json',orient='index')

# Analyses de données


In [9]:
import ipywidgets as widgets
from ipywidgets import GridspecLayout, Image, interact
import random
from IPython.display import display
import pandas as pd

# === chargement des données depuis le fichier JSON ===
df = pd.read_json('donnees.json', orient='index')
#ajout d'une colonne pour la variable de préférence
df['preferences'] = None


# Function to get selected images
def get_selected_images(btn):
    # Récupération des identifiants des images sélectionnées
    selected_ids = [list_id[i] for i, checkbox in enumerate(checkboxes) if checkbox.value]
    # traitement des préférences
    with output:
        output.clear_output()
        for id in list_id:
            if id in selected_ids:
                preferences(id, 2)  # Appel de la fonction preferences avec la valeur 1
            else:
                preferences(id, -1)  # Appel de la fonction preferences avec la valeur -0.5
        
        # enregistrement du DataFrame dans le fichier JSON
        print("Données enregistrés dans le fichier JSON")
        df.to_json('donnees.json',orient='index')
        
    
    
def preferences(image_id, valeur):
    # Mise à jour de la variable de préférence
    df.at[image_id, 'preferences'] = valeur
    


list_id = []
# liste d'identifiants d'images aléatoires (pas de doublons)
while len(list_id) < 50:
    id = random.randint(0, len(df) - 1)
    if id not in list_id:
        list_id.append(id)
               
    
checkboxes = [widgets.Checkbox(value=False, description='Aime') for _ in range(len(list_id))]

# Create the GridspecLayout widget
layout = GridspecLayout(n_columns=2, n_rows=len(list_id), width='400px')
for i, (id, checkbox) in enumerate(zip(list_id, checkboxes)):
    image = open(f"images/images/{df.iloc[id]['name']}.{df.iloc[id]['img_type']}", "rb").read()
    image_widget = widgets.Image(
        value=image,
        format='png',
        width=100,
        height=100,
    )
    layout[i,0] = image_widget
    layout[i, 1] = checkbox

# Button to get selected images
button_select = widgets.Button(description="Select")

# Output widget to display selected images
output = widgets.Output()


# Link button click event to function
button_select.on_click(get_selected_images)

# Display the layout and button
display(layout, button_select, output)


GridspecLayout(children=(Image(value=b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x00x\x00\x00\x00x\x08\x03\x…

Button(description='Select', style=ButtonStyle())

Output()

# Visualisation des données

In [ ]:
pass

# Système de recommandation

In [ ]:
pass

# Tests

pass